In [1]:
pip install sentence-transformers pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import json

In [3]:
with open("iso27002_extracted.json", "r", encoding="utf-8") as f:
    iso_data = json.load(f)

with open("cleaned_sopsv2.json", "r", encoding="utf-8") as f:
    sop_data = json.load(f)

In [10]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
sop_text = sop_data[4]["text"]
results = []


In [12]:
for clause_id, content in iso_data.items():
    clause_name = content.get("Clause Name", "").strip()
    control = content.get("Control", "").strip()
    purpose = content.get("Purpose", "").strip()
    guidance = content.get("Guidance", "").strip()

    if not (control or purpose or guidance):
        continue

    full_text = f"{control} {purpose} {guidance}".strip()

    embeddings = model.encode([full_text, sop_text])

    score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

    results.append({
        "Clause ID": clause_id,
        "Clause Name": clause_name,
        "Similarity Score": round(score, 3),
        "Remarks": "Relevant" if score > 0.5 else "Not Relevant"
    })

In [13]:
df = pd.DataFrame(results).sort_values(by="Similarity Score", ascending=False).reset_index(drop=True)

In [14]:
df.to_csv("ISO_to_SOP_Similarity_Report_4_1.csv", index=False)
print(df.head(20))

   Clause ID                                        Clause Name  \
0       5.27      Learning fr om information security incidents   
1       5.26        R esponse to information security incidents   
2       5.24  Information secur ity incident management plan...   
3       5.33                             Pr otection of records   
4       5.35        Independent r eview of information security   
5       5.36  C ompliance with policies, rules and standards...   
6       5.21  Managing informa tion security in the ICT supp...   
7       5.22  Monit oring, review and change management of s...   
8       5.37                   Document ed operating procedures   
9       5.34                     Pri vacy and protection of PII   
10       6.5  R esponsibilities after termination or change ...   
11      5.23    Information secur ity for use of cloud services   
12       6.4                              Disciplinary pr ocess   
13      5.31  Leg al, statutory, regulatory and contractual ..